In [2]:
import pandas as pd
from utilities import clean_data

In [3]:
df, df_brands, df_allbrands, brands, compsets, compset_groups, groups_bycompset = clean_data()

In [4]:
df.head()

,period_end_date,business_entity_doing_business_as_name,followers,pictures,videos,comments,likes
239152,2017-05-06,24S,-1.0,-1.0,-1.0,-1.0,-1.0
239162,2017-05-13,24S,-1.0,6.0,3.0,57.0,1765.0
239538,2017-05-20,24S,-1.0,6.0,3.0,57.0,1765.0
239200,2017-05-27,24S,-1.0,6.0,3.0,57.0,1765.0
239161,2017-06-03,24S,-1.0,24.0,3.0,109.0,3922.0


In [5]:
df_brands.head()

,business_entity_doing_business_as_name,compset_group,compset,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name
0,24S,Apparel Retail,"[Apparel Retail, Ecommerce Apparel Retail, Eco...",LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris
1,3.1 Phillip Lim,Luxury & Premium & Mainstream,"[Luxury & Premium & Mainstream, Soft Luxury, U...",Alexander Wang Inc,None,Alexander Wang Inc,None
2,3CE,Beauty & Boutique,[Beauty & Boutique],L'Oreal,France,L'Oreal,Euronext Paris
3,A. Lange & Soehne,Luxury & Premium & Mainstream,"[Hard Luxury, Luxury & Premium & Mainstream]",Richemont,Switzerland,Richemont,Six Swiss Exchange
4,ANIMALE,Apparel Retail,[Apparel Retail],GRUPO DE MODA SOMA SA,Brazil,GRUPO DE MODA SOMA SA,Bolsa de Valores de Sao Paulo (XBSP)


In [6]:
df_allbrands.head()

,period_end_date,compset_group,compset,followers,pictures,videos,comments,likes
365993,2019-01-05,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,577822196.0,10851.0,1726.0,711070.0,110228425.0
365994,2020-10-10,Apparel Retail,Omnichannel Apparel Retail,374077484.0,13141.0,1352.0,659640.0,71433543.0
365995,2021-05-15,Fitness & Exercise,Fitness & Exercise,12247138.0,2207.0,570.0,81662.0,1814642.0
365996,2023-05-27,Food Products,Snack Bars,2053344.0,147.0,81.0,56011.0,48641.0
365997,2023-05-06,Food Products,Food Products,8001015.0,538.0,439.0,71680.0,258727.0


In [7]:
compsets

array(['Soft Luxury', 'Luxury & Premium & Mainstream',
       'US Softlines Analyst Interest List',
       'Global Luxury Analysts Interest List', 'Coffee', 'Restaurants',
       'Beverages', 'Sports drinks', 'Energy drinks', 'Fast Fashion',
       'Omnichannel Apparel Retail', 'Apparel Retail',
       'US Department Store', 'Latam Apparel Retail', 'Food Products',
       'Snack Bars', 'Pasta Sauce', 'Ecommerce Apparel Retail',
       'Ecommerce Mainstream Apparel Retail', 'Sporting Goods',
       'Casual Dining', 'Hard Luxury', 'Sportswear & Athleisure',
       'Athleisure', 'Footwear', 'Beauty & Boutique', 'Building Products',
       'Sportswear', 'Petcare', 'Premium Brands', 'QSR', 'Alcohol',
       'Toys & Collectibles', 'Mattress', 'Fitness & Exercise',
       'US Discount Store', 'Ecommerce Luxury Apparel Retail',
       'Plant-Based Meat', 'Food Retail', 'Discount Retailers ',
       'Mid-Range Watch & Jewelry', 'Outdoor Gear', 'Plant-Based Dairy',
       'Mainstream Brands', 'D

In [8]:
compset_groups

array(['Luxury & Premium & Mainstream', 'Restaurants', 'Beverages',
       'Apparel Retail', 'Food Products', 'Sporting Goods',
       'Sportswear & Athleisure', 'Beauty & Boutique',
       'Building Products', 'Petcare', 'Toys & Collectibles', 'Mattress',
       'Fitness & Exercise', 'Food Retail', 'Discount Retailers ',
       'Outdoor Gear', 'Dermatology and Orthodontics', 'Entertainment',
       'Home Appliances'], dtype=object)

In [9]:
groups_bycompset

{'Soft Luxury': array(['Luxury & Premium & Mainstream'], dtype=object),
 'Luxury & Premium & Mainstream': array(['Luxury & Premium & Mainstream'], dtype=object),
 'US Softlines Analyst Interest List': array(['Luxury & Premium & Mainstream', 'Apparel Retail',
        'Sportswear & Athleisure', 'Outdoor Gear'], dtype=object),
 'Global Luxury Analysts Interest List': array(['Luxury & Premium & Mainstream'], dtype=object),
 'Coffee': array(['Restaurants'], dtype=object),
 'Restaurants': array(['Restaurants'], dtype=object),
 'Beverages': array(['Beverages'], dtype=object),
 'Sports drinks': array(['Beverages'], dtype=object),
 'Energy drinks': array(['Beverages'], dtype=object),
 'Fast Fashion': array(['Apparel Retail'], dtype=object),
 'Omnichannel Apparel Retail': array(['Apparel Retail'], dtype=object),
 'Apparel Retail': array(['Apparel Retail'], dtype=object),
 'US Department Store': array(['Apparel Retail'], dtype=object),
 'Latam Apparel Retail': array(['Apparel Retail'], dtype=obje

In [11]:
df_allbrands_total = df_allbrands.copy()
df_allbrands_total = df_allbrands_total[df_allbrands_total['compset'] == 'Study (All Brands)']
df_allbrands_total

,period_end_date,compset_group,compset,followers,pictures,videos,comments,likes
366010,2018-09-08,Study (All Brands),Study (All Brands),1.773277e+09,87181.0,14673.0,7314582.0,597258286.0
366031,2016-04-02,Study (All Brands),Study (All Brands),6.008734e+08,67359.0,6009.0,4709693.0,435171951.0
366032,2019-11-09,Study (All Brands),Study (All Brands),2.419067e+09,92488.0,17796.0,6909332.0,537764660.0
366052,2018-10-20,Study (All Brands),Study (All Brands),1.830406e+09,90080.0,16058.0,8696711.0,591024307.0
366064,2015-11-28,Study (All Brands),Study (All Brands),4.454604e+08,49790.0,3111.0,2789593.0,260657473.0
...,...,...,...,...,...,...,...,...
393227,2022-06-25,Study (All Brands),Study (All Brands),3.212268e+09,75288.0,24851.0,4620627.0,255648642.0
393295,2020-11-28,Study (All Brands),Study (All Brands),2.760049e+09,141746.0,22342.0,14073115.0,460119978.0
393463,2017-11-04,Study (All Brands),Study (All Brands),1.266705e+09,76883.0,13525.0,4300920.0,568469222.0
393516,2016-07-09,Study (All Brands),Study (All Brands),7.209781e+08,69044.0,7160.0,3044215.0,449559655.0


In [17]:
#Normalization of Data to Data from All Brands
df_normalized = df.copy()
# Convert the 'period_end_date' column to datetime objects
df_normalized['period_end_date'] = pd.to_datetime(df_normalized['period_end_date'])

# Convert the 'period_end_date' column to numeric
df_normalized['period_end_date_numeric'] = df_normalized['period_end_date'].astype(int)

for date in df_allbrands['period_end_date']:
    # Normalize each column in df_normalized by the corresponding value in df_allbrands_total
    df_normalized[df_normalized['period_end_date'] == date]['followers'] /= df_allbrands_total[df_allbrands_total['period_end_date'] == date]['followers']
    df_normalized[df_normalized['period_end_date'] == date]['pictures']  /= df_allbrands_total[df_allbrands_total['period_end_date'] == date]['pictures']
    df_normalized[df_normalized['period_end_date'] == date]['videos']    /= df_allbrands_total[df_allbrands_total['period_end_date'] == date]['videos']
    df_normalized[df_normalized['period_end_date'] == date]['comments']  /= df_allbrands_total[df_allbrands_total['period_end_date'] == date]['comments']
    df_normalized[df_normalized['period_end_date'] == date]['likes']     /= df_allbrands_total[df_allbrands_total['period_end_date'] == date]['likes']

/var/folders/f8/pknr7qxj61j1p2c5nnq5qmbm0000gn/T/ipykernel_7717/1539764222.py:7: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df_normalized['period_end_date_numeric'] = df_normalized['period_end_date'].astype(int)
/var/folders/f8/pknr7qxj61j1p2c5nnq5qmbm0000gn/T/ipykernel_7717/1539764222.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normalized[df_normalized['period_end_date'] == date]['followers'] /= df_allbrands_total[df_allbrands_total['period_end_date'] == date]['followers']
/var/folders/f8/pknr7qxj61j1p2c5nnq5qmbm0000gn/T/ipykernel_7717/1539764222.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [ ]:
plt.figure(figsize=(18,12))
plt.plot(df_normalized[df_normalized['business_entity_doing_business_as_name'] == '24S']['period_end_date'],
         df_normalized[df_normalized['business_entity_doing_business_as_name'] == '24S']['followers'])
plt.show()